In [1]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

In [2]:
import pandas as pd

In [19]:
df_train = pd.read_csv('products_sentiment_train.tsv', sep = '\t', header = None)
df_test = pd.read_csv('products_sentiment_test.tsv', sep = '\t')

In [185]:
texts = df_train.iloc[:, 0].to_list()
labels = df_train.iloc[:, 1].to_list()

In [225]:
vectorizers = [CountVectorizer(), TfidfVectorizer()]
classifiers = [LogisticRegression(), SGDClassifier(), LinearSVC()]
for vectorizer in vectorizers:
    for classifier in classifiers:
        clf = Pipeline([('vectorizer', vectorizer),('classifier', classifier)])
        print(clf)
        print(cross_val_score(clf, preproc_my1(texts1), labels).mean())
        

Pipeline(memory=None,
         steps=[('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('classifier',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='auto', n_jobs=None,
             

In [75]:
from sklearn.model_selection import GridSearchCV

In [78]:
param_grid = {
    'vectorizer__ngram_range': [(1, 2), (1, 3)], 'vectorizer__min_df' : [0.1, 1, 5, 10], 'vectorizer__sublinear_tf' : [False, True],
    'classifier__C' : [0.8, 0.9, 2, 10]
}
clf = Pipeline(
    [('vectorizer', TfidfVectorizer(ngram_range = (1, 2), min_df = 1, sublinear_tf = True)),
    ('classifier', LinearSVC(C = 0.8))]
    )
search = GridSearchCV(clf, param_grid, n_jobs=-1)
search.fit(texts, labels)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

Best parameter (CV score=0.790):
{'classifier__C': 0.8}


In [228]:
best_model = Pipeline(
    [('vectorizer', TfidfVectorizer(analyzer = 'word', ngram_range = (1, 2), sublinear_tf = True)),
    ('classifier', LinearSVC(C = 0.8))]
    )

In [217]:
from nltk.stem.snowball import SnowballStemmer
englishStemmer=SnowballStemmer("english")

'have'

In [226]:
def preproc_my(texts):
    res = []
    for text in texts:
        text = nltk.tokenize.word_tokenize(text)
        res.append(' '.join([englishStemmer.stem(word) for word in text]))
    return(res)

In [229]:
cross_val_score(best_model, preproc_my(texts), labels).mean()

0.7905

In [231]:
best_model.fit(preproc_my(texts), labels)

Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=True,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('classifier',
                 LinearSVC(C=0.8, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
    

In [245]:
df_test['y'] = (best_model.predict(df_test.loc[:, 'text'].to_list()))

In [247]:
df_test[['Id','y']].to_csv('product-reviews-sentiment-analysis-light.csv', index = False)

# word2vec

In [248]:
text2word = [nltk.tokenize.word_tokenize(text) for text in texts] 

In [249]:
w2model = models.Word2Vec(text2word)
words = list(w2model.wv.vocab)
print(words)

['2', '.', 'take', 'around', 'pictures', 'i', 'downloaded', 'a', 'version', 'of', 'computer', 'and', 'antivirus', 'in', 'love', 'with', 'security', 'system', 'all', 'over', 'again', 'the', 'wrt54g', 'plus', 'is', 'perfect', 'if', 'you', 'need', 'wireless', 'area', 'or', 'for', 'house', 'as', 'was', 'my', 'case', 'dont', 'especially', 'like', 'how', 'music', 'files', 'are', ';', 'basically', 'they', 'just', 'into', 'one', 'folder', 'no', ',', 'might', 'have', 'windows', 'explorer', 'folders', 'using', 'pail', '...', 'it', 'worked', 'ok', 'until', 'device', 'can', 'your', 'change', 'make', 'different', 'type', 'display', 'either', 'list', ')', 'between', 'songs', '-', 'less', 'than', 'that', 'on', 'other', 'devices', 'good', 'based', 'player', 'from', 'internet', 'hitachi', 'm12v', '/', '4', 'router', 'time', 'came', 'up', 'to', 'be', 'choice', 'best', 'its', 'price', 'loved', 'this', 'product', 'when', 'first', 'got', 'great', 'sound', 'easy', 'use', '(', 'even', 'though', 'software', "

In [250]:
w2v = dict(zip(w2model.wv.index2word, w2model.wv.syn0))

/Users/georg/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


In [149]:
import numpy as np

In [251]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = len(word2vec)

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([np.mean([self.word2vec[w] for w in words if w in self.word2vec] 
                        or [np.zeros(self.dim)], axis=0) for words in X ])


In [252]:
MeanEmbeddingVectorizer(w2v).transform(text2word)

array([[ 0.08867643, -0.19030951,  0.07673012, ..., -0.01103395,
        -0.09983047,  0.12514542],
       [ 0.08527805, -0.1810464 ,  0.07251785, ..., -0.00963403,
        -0.09875368,  0.11876567],
       [ 0.09658543, -0.20234562,  0.08120908, ..., -0.00809703,
        -0.1082152 ,  0.13236675],
       ...,
       [ 0.10639463, -0.22192523,  0.09260677, ..., -0.0085217 ,
        -0.11881649,  0.14600149],
       [ 0.0911987 , -0.1886365 ,  0.07591707, ..., -0.00805273,
        -0.10253359,  0.12425323],
       [ 0.09279472, -0.19538929,  0.08052409, ..., -0.00816769,
        -0.10395259,  0.12858798]], dtype=float32)

In [253]:
from sklearn.ensemble import ExtraTreesClassifier

In [256]:
Extra_Trees_w2v = Pipeline([
    ("word2vec vectorizer", MeanEmbeddingVectorizer(w2v)),
    ("extra trees", LogisticRegression())])

In [257]:
cross_val_score(Extra_Trees_w2v, text2word, labels)

array([0.635 , 0.6375, 0.6325, 0.6325, 0.635 ])